
# 📊 Proyecto Flask + Google Sheets + Power BI

Paso a paso proyecto en **Python + Flask** que:
- Maneja inicio de sesión de usuarios
- Registra accesos en **Google Sheets**
- Muestra un **Dashboard de Power BI**
- Permite descargar un archivo Excel

El objetivo es **explicar el código por secciones**, facilitando su mantenimiento y aprendizaje.


## 📦 Importación de librerías

In [ ]:

import os
import time
import gspread
from google.oauth2.service_account import Credentials
from flask import Flask, render_template, request, redirect, url_for, session, send_from_directory
from datetime import datetime, timedelta


## 🌎 Configuración de Zona Horaria

In [ ]:

os.environ['TZ'] = 'America/Mexico_City'
time.tzset()


## ⚙️ Configuración de Flask

In [ ]:

app = Flask(__name__)
app.secret_key = os.environ.get("SECRET_KEY", "keytronics123")
app.permanent_session_lifetime = timedelta(minutes=30)


## 📄 Configuración de Google Sheets

In [ ]:

GOOGLE_SHEETS_ID = os.environ.get(
    "GOOGLE_SHEETS_ID",
    "14x3JapIuLdgclmk4_ls3NCMWZVJF5fjNhyIGI6G86dE"
)

CREDENCIALES_JSON = "credenciales_google.json"
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]


## 🔐 Autenticación con Google Sheets

In [ ]:

sheet = None

try:
    google_credentials_env = os.environ.get("GOOGLE_CREDENTIALS")

    if google_credentials_env:
        import json
        creds_info = json.loads(google_credentials_env)
        credentials = Credentials.from_service_account_info(
            creds_info, scopes=SCOPES
        )
        print("Credenciales cargadas desde ENV")
    else:
        credentials = Credentials.from_service_account_file(
            CREDENCIALES_JSON, scopes=SCOPES
        )
        print("Credenciales cargadas desde archivo local")

    client = gspread.authorize(credentials)
    sheet = client.open_by_key(GOOGLE_SHEETS_ID).sheet1
    print("Conexión exitosa con Google Sheets")

except Exception as e:
    print("Error conectando con Google Sheets:", e)
    sheet = None


## 👥 Usuarios permitidos

In [ ]:

USUARIOS = {
    "Jose_Consultor": "654321987",
    "AlfonsoCampo": "N7$vL8qY#x3B",
    "AlejandroCampo": "tR!5mK2wQ9#z",
    "DavidVargas": "Fp8#Vd4!sZ1q",
    "BrendaMuñoz": "H2!xW7qR#k9L"
}


## 📝 Función para registrar accesos

In [ ]:

def registrar_acceso(usuario):
    if not sheet:
        print("No hay conexión con Google Sheets")
        return

    fecha = datetime.now().strftime("%Y-%m-%d")
    hora = datetime.now().strftime("%H:%M:%S")

    sheet.append_row([usuario, fecha, hora])
    print(f"Acceso registrado: {usuario} {fecha} {hora}")


## 🔐 Rutas de Login

In [ ]:

@app.route("/")
def home():
    return render_template("login.html")

@app.route("/login", methods=["POST"])
def login():
    username = request.form["username"]
    password = request.form["password"]

    if username in USUARIOS and USUARIOS[username] == password:
        session.permanent = True
        session["user"] = username
        registrar_acceso(username)
        return redirect(url_for("dashboard"))

    return render_template("login.html", error="Usuario o contraseña incorrectos")


## 📊 Dashboard con Power BI

In [ ]:

@app.route("/dashboard")
def dashboard():
    if "user" not in session:
        return redirect(url_for("home"))

    embed_url = "https://app.powerbi.com/view?r=TU_URL_AQUI"

    return f"<iframe src='{embed_url}' width='100%' height='800'></iframe>"


## 📥 Descarga de archivo

In [ ]:

@app.route("/descargar_csv")
def descargar_csv():
    if "user" not in session:
        return redirect(url_for("home"))

    directorio = os.path.dirname(os.path.abspath(__file__))
    archivo = "datos_reporte_Keytronics.xlsx"

    return send_from_directory(directorio, archivo, as_attachment=True)


## 🚪 Logout

In [ ]:

@app.route("/logout")
def logout():
    session.pop("user", None)
    return redirect(url_for("home"))


## ▶️ Ejecución del servidor

In [ ]:

if __name__ == "__main__":
    port = int(os.environ.get("PORT", 10000))
    app.run(host="0.0.0.0", port=port, debug=False)
